# <font color='red'> Predicting Time Series - ARIMA & SARIMAX </font>

In this module we'll explore two very popular models for time series forecasting - ARIMA and SARIMAX (don't worry if they just sound like pokemon at this point - we'll break down the acronyms).

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(rc={'figure.figsize':(14,8)})

import warnings
warnings.simplefilter("ignore")

We load the tractor sales dataset

In [ ]:
sales = pd.read_csv('data/Tractor-Sales.csv')['Number of Tractor Sold']
sales.head()

We try to turn the timeseries stationary by logging and diffing

In [ ]:
sales_logged = sales.map(np.log)
sales_logged_diff = sales_logged.diff().dropna()

In [ ]:
sales_logged_diff.plot();

## <font color="red"> ARIMA </font>

[ARIMA](http://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_model.ARIMA.html) is one of the most used models for predicting timeseries. There are more sophisticated versions that we will talk later, but for now we'll use the plain vanilla version to develop some intuitions. 

ARIMA stands for: 

- (AR) Auto Regressive 
- (I)  Integrated 
- (MA) Moving Average

The names are a bit misleading, let's go into them in more detail. 

About the model API, it elegantly depends only on 3 parameters: 

> `ARIMA(p,d,q)`

Notes: 
- _The results of our predictions in this notebook will suck, as we are trying to avoid adding complexity, and showcase the API._ 
- _If you want to try the hardcore version, [this article](https://people.duke.edu/~rnau/411arim.htm) has a very well made explanation for many cases of ARIMA. We are, as mentioned, going to stick to the super-basic "get your hands dirty" ARIMA._
- _If you are feeling scared with how to select hyper parameters for ARIMA remember that in the real world hyper parameter optimizers can take care of most of this. This notebook shows things as they were in the time where programmers had nothing but a dusty book, a magnetized needle and a steady hand. However getting these intuitions will help you debug and explain your models later._ 

### <font color='red'> Auto Regressive </font>

The first of our 3 parameters, `p`, is the "number of auto-regressive terms". 

What are auto-regressive terms? They are quite simply the lags of the dependent variable. Is the present point dependent on the previous one? On the previous two? Eight? 

In plain English, the auto-regressive model says that: 
> “The value at a particular time depends on the value at the previous times (+ error)” 

To use this model (we'll use just an AR model, and then add the other components to see the difference), we need to choose the parameter `p`, and use the model as `ARIMA(p, 0, 0)`. 

<font color='red'> **Choosing the hyper-parameter `p`** </font>

Choosing the parameter `p`, as with just about everything in data science, is a mixture of heuristics and experience. 

Sometimes, you have a strong business reason to say "I want to only use the previous p lags". In others, you want to use the data to tell you. 

Given that in this case we don't have much of a business intuition about tractor sales (at least I don't), we will use the PACF (partial auto-correlation function) to determine a suitable value. 

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import pacf
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf

import utils

In [ ]:
plot_pacf(sales_logged_diff, alpha=.05, lags=40, method='ywmle')  
sns.mpl.pyplot.xlabel('lag')
sns.mpl.pyplot.ylabel('Autocorrelation');

What does this tell us? What we would like to see is a few components clearly go over the significance line. 

A rule of thumb is to start by testing the number of components where the significance line gets crossed, and make that our `p`. This example was chosen to show that sometimes this doesn't actually happen, and that sometimes these cases are a lot more ambiguous. 

So we have no guarantees that the data contains an AR process (there is some indication it doesn't), but let's experiment with the ARIMA model to see if we can spot any pattern in the AR part: 

In [ ]:
model = ARIMA(sales_logged_diff.values, order=(1, 0, 0))
results = model.fit()
results

A few of things to note about the API:
1. We passed the data straight to the model (unlike in sklearn) 
2. We used `.values` to get the numpy array instead of the pandas series
3. We passed the order `(p, d, q)` as `(1, 0, 0)`, as we'd established that we want to try `p=1`
4. We called `fit` without any parameters (which is also different from sklearn) 
5. The model isn't fit in place, we have to grab the results with a results

In [ ]:
results.plot_predict();

### <font color='red'> Integrated </font>

The "integrated" part of the name simply means that we take the diff between consecutive periods to make the time series stationary. We've already done this ahead of time (because we needed it for our ACF and PACF plots), but you can also leave it as a hyper parameter and tune it later. 

Anyway, to change the parameter `d`, we follow the same logic as before. Let's set `p` to 1, and try a few values of `d`: 

In [ ]:
def try_parameter_d(data, d):
    model = ARIMA(data, order=(1, d, 0))
    results = model.fit()
    results.plot_predict()
    sns.mpl.pyplot.title(f"ARIMA with d={d}")
    sns.mpl.pyplot.show()

In [ ]:
# notice that we are passing the original logged data, and letting d diff it as it needs
try_parameter_d(sales_logged.astype(float).values, 0)
try_parameter_d(sales_logged.astype(float).values, 1)
try_parameter_d(sales_logged.astype(float).values, 2)

### <font color='red'>Moving average </font>

The last of our 3 parameters, `q`, is the "number of moving average terms". 

The logic here is similar to the one we used for `p`, but instead of predicting values with lagged values, we are predicting errors with lagged errors. 

The MA terms are lagged forecast errors. In this model, what predicts `x(t)` is `e(t-1)`, `e(t-2)`, ..., where `e(i)` is the difference between the moving average at the ith instant and the actual value.

In plain English, the moving average model says that: 
> _“your function at a particular time is your error at that time, plus a parameter theta times your error at a previous time”_

<font color='red'> **Choosing the hyper-parameter `q`** </font>

The rule of thumb for setting `q` is to use the ACF. The reasons are not trivial, and are well explained in [this fantastic StackExchange post](https://stats.stackexchange.com/questions/281666/how-does-acf-pacf-identify-the-order-of-ma-and-ar-terms?utm_medium=organic&utm_source=google_rich_qa&utm_campaign=google_rich_qa), if you are interested in digging deeper. 

If not, just remember: 
> For `p` use the PACF  
> For `q` use the ACF. 

So, let's plot our ACF: 

In [ ]:
plot_acf(sales_logged_diff, alpha=.05, lags=40)  
sns.mpl.pyplot.xlabel('lag')
sns.mpl.pyplot.ylabel('Autocorrelation');

As before, the rule of thumb is "where we first cross the significance line", so we'll try q=1

In [ ]:
model = ARIMA(sales_logged_diff.values, order=(0, 0, 1))
results = model.fit()
results.plot_predict();

### <font color='red'> Putting the ARIMA components together </font>

Great, so now we know how to use the Auto Regressive model, the Moving Average model, and how to control how much differentiation goes on. It only remains to put it all together, and have some fun. 

In [ ]:
model = ARIMA(sales_logged_diff.dropna().values, order=(1, 1, 1))
results = model.fit()
results.plot_predict();

In [ ]:
model = ARIMA(sales_logged_diff.dropna().values, order=(5, 2, 1))
results = model.fit()
results.plot_predict();

In [ ]:
model = ARIMA(sales_logged_diff.dropna().values, order=(0, 1, 0))
results = model.fit()
results.plot_predict();

## <font color='red'> Undoing the transformations <font>

One last, but critical point, is how to undo our transformations. As you might have noticed, the plots we made don't actually predict sales of tractors, they predict... some weird logged thing differentiated twice. 

### <font color='red'>Undoing the diff </font>

How can we undo our diff? To take a metaphor from the brilliant [Aileen Nielsen](https://www.youtube.com/watch?v=zmfe2RaX-14), _"If I tell you my house is 200m shorter than the empire state building, you don't know the height of either"_. 

Let's try to undo the diff, with a simple example: 

In [ ]:
a = pd.Series([2, 6, 4, 6, 2,])
a

In [ ]:
a_diff = a.diff()
a_diff

Lost the first term, as we know. Now, let's take the [cumulative sum](http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.cumsum.html) of `a_diff`

In [ ]:
a_diff_cumsum = a_diff.cumsum()
a_diff_cumsum

Great, now let's fill that first point with zero, and add the first element of the original series to everything: 

In [ ]:
rebuilt = a_diff_cumsum.fillna(0) + 2
rebuilt

Did that work? 

In [ ]:
rebuilt == a

In [ ]:
def rebuild_diffed(series, first_element_original):
    
    # get the cumulative sum
    cumsum = series.cumsum()
    
    # fill the most recent zero 
    # (in our case there was just one, but this way it works beyond it)
    most_recent_null = cumsum.loc[cumsum.isnull()].index.max()
    cumsum.loc[most_recent_null] = 0 
    
    # return the cumsum, plus the first original element
    return cumsum + first_element_original

In [ ]:
rebuild_diffed(a_diff, 2)

### <font color='red'> Undoing the log  </font>

Undoing the log is the easy part, it's simply by taking the exponent. 

In [ ]:
a

In [ ]:
logged = a.map(np.log)
logged

In [ ]:
rebuilt = logged.apply(np.exp)
rebuilt

In [ ]:
rebuilt == a

Excellent!, let's rebuild our own predictions

In [ ]:
model = ARIMA(sales_logged_diff.dropna().values, order=(1, 1, 1))
results = model.fit();

# Get the series of results. These are our un-transformed predictions 
predictions_after_one_diff_and_a_log = pd.Series(results.predict())

# Re-build the logged predictions from the diff of the logged predictions 
predictions_after_a_log = rebuild_diffed(predictions_after_one_diff_and_a_log, 
                                                      sales_logged.dropna().iloc[0])

# Re-build the predictions from the log of the predictions 
predictions = predictions_after_a_log.map(np.exp)

In [ ]:
predictions_after_one_diff_and_a_log.plot();

In [ ]:
predictions.plot(label="forecast")
sales.plot(label="y")
sns.mpl.pyplot.legend();

## <font color='red'> SARIMAX </font>

SARIMAX stands for **Seasonal Autoregressive Integrated Moving Average with Exogeneous Regressors**, and is an evolution of ARIMA, that takes into account seasonality.

The **Autoregressive Integrated Moving Average** part we just saw, what about the new bits? 

- **`Seasonal`**: as the name suggests, this model can actually deal with seasonality. Coool.... 
- **`With Exogenous`** roughly means we can add external information. For instance we can include the temperature time series to predict the ice cream sales, which is surely useful. Exogenous variables are introduced from or produced outside the organism or system, and don't change with the predictions of the system.

What are the parameters? 

These we already know:
> p = 0  
> d = 1  
> q = 1  

But now we have a second bunch. The first 3 are the same as before, but for the seasonal part: 
> P = 1  
> D = 1  
> Q = 1  

The last new parameter, `S`, is an integer giving the periodicity (number of periods in season). 
We normally have a decent intuition for this parameter: 
- If we have daily data and suspect we may have weekly trends, we may want `S` to be 7. 
- If the data is monthly and we think the time of the year may count, maybe try `S` at 12 
> S = 12   

To know the SARIMAX in detail you can take a closer look at [the documentation](http://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.sarimax.SARIMAX.html).

In [ ]:
import statsmodels.api as sm

We load the airfline passengers dataset

In [ ]:
airlines = utils.load_airline_data()
airlines = airlines[:'1957']

In [ ]:
airlines.plot();

In [ ]:
model = sm.tsa.statespace.SARIMAX(airlines,             
                          order=(0, 1, 1),              
                          seasonal_order=(1, 1, 1, 12)) # <-- We'll get into how we found these
model

In [ ]:
results = model.fit()

easy to get predictions!

In [ ]:
results.get_prediction?

In [ ]:
pred = results.get_prediction(
                            start=airlines.index.min(),  # <--- Start at the first point we have 
                            dynamic=False)               # <--- predict the last periods without data

In [ ]:
mean_predictions = pred.predicted_mean
mean_predictions.head(10)

In [ ]:
airlines.plot(label='observed')
mean_predictions.plot(label='One-step ahead Forecast with dynamic=False', alpha=.7)
sns.mpl.pyplot.legend();

We can check the confidence intervals:

In [ ]:
pred_ci = pred.conf_int()
pred_ci.head(10)

We can use matplotlib ugly `fill_between` to plot the confidence intervals

In [ ]:
airlines.plot(label='observed')
mean_predictions.plot(label='One-step ahead Forecast with dynamic=False', alpha=.7)

# Let's use some matplotlib code to fill between the upper and lower confidence bound with grey 
sns.mpl.pyplot.fill_between(pred_ci.index, 
                 pred_ci['lower passengers_thousands'],
                 pred_ci['upper passengers_thousands'], 
                 color='k', 
                 alpha=.2)

sns.mpl.pyplot.ylim([0, 700])
sns.mpl.pyplot.legend();

Kind of makes sense, at the start we didn't have enough data to predict much, so the uncertaintly band is pretty insane.

In [ ]:
def plot_predictions(series_, pred_):
    
    """ 
    Utility function to plot time series predictions with Confidence intervals
    """
    mean_predictions_ = pred_.predicted_mean
    pred_ci_ = pred_.conf_int()   
    series_.plot(label='observed')
    mean_predictions_.plot(label='predicted', 
                           alpha=.7)

    sns.mpl.pyplot.fill_between(pred_ci_.index,
                     pred_ci_['lower passengers_thousands'],
                     pred_ci_['upper passengers_thousands'], 
                     color='k', 
                     alpha=.2)

    sns.mpl.pyplot.ylim([0, 700])
    sns.mpl.pyplot.legend()
    sns.mpl.pyplot.show()

Now, what if we had stopped feeding it data, and asked it to predict the last 30 periods?

In [ ]:
# We want to make 30 steps out of time 
train_up_to_step = len(airlines) - 30

# remember the dynamic argument? Well, we'll use the first 30 steps to train
pred = results.get_prediction(start=airlines.index.min(),  
                              dynamic=train_up_to_step)                     
    
plot_predictions(series_=airlines, pred_=pred)

Pretty cool, we can see the uncertainty increasing as we move. Also, what if we wanted to forecast outside of our "known" dates?

For this we will use the `get_forecast()` method, which allows us to go beyond what data we have:

In [ ]:
forecast = results.get_forecast(steps=15)
forecast_ci = forecast.conf_int()

In [ ]:
plot_predictions(series_=airlines, pred_=forecast)